## Obtención de imagenes

In [ ]:
import os
import time
import requests
import io
import hashlib
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATASET_PATH = os.path.join(PROJECT_ROOT, 'dataset')

Creamos las funciones para crear una carpeta y para descargar las imagenes

In [ ]:
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directorio '{directory}' creado exitosamente.")
    else:
        print(f"El directorio '{directory}' ya existe.")

def is_valid_image(image_content):

    try:
        Image.open(io.BytesIO(image_content))
        return True
    except:
        return False

def download_image(url, folder_path, img_name):
    try:
        # Configurar headers para simular un navegador
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, stream=True, timeout=10)

        if response.status_code == 200:
            content = response.content

            # Verificar que sea una imagen válida
            if is_valid_image(content):
                # Guardar la imagen
                with open(os.path.join(folder_path, f"{img_name}.jpg"), 'wb') as f:
                    f.write(content)
                return True

        return False
    except Exception as e:
        print(f"Error al descargar imagen: {str(e)}")
        return False

Creamos la función para extraer las urls usando bing.

In [ ]:
def get_image_urls_using_bing(query, max_links_to_fetch=100):
    """Obtiene URLs de imágenes desde Bing."""
    search_url = f"https://www.bing.com/images/search?q={query.replace(' ', '+')}&form=HDRSC2&first=1"

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(search_url)
        time.sleep(2)  # Esperar carga inicial

        # Scroll progresivo para cargar más imágenes
        scroll_pause_time = 3  # Tiempo entre scrolls
        screen_height = driver.execute_script("return window.innerHeight")
        scrolls = 0
        image_urls = set()  # Usar set para evitar duplicados

        while len(image_urls) < max_links_to_fetch and scrolls < 20:  # Máximo 20 scrolls
            # Scroll hacia abajo
            driver.execute_script(f"window.scrollBy(0, {screen_height * 2});")
            time.sleep(scroll_pause_time)

            # Capturar nuevas imágenes
            thumbnails = driver.find_elements(By.CSS_SELECTOR, ".mimg")

            for img in thumbnails:
                if len(image_urls) >= max_links_to_fetch:
                    break
                try:
                    # Obtener URL real desde data-src (evitar placeholders)
                    url = img.get_attribute("data-src") or img.get_attribute("src")
                    if url and url.startswith("http") and url not in image_urls:
                        image_urls.add(url)
                except StaleElementReferenceException:
                    continue

            scrolls += 1

        return list(image_urls)[:max_links_to_fetch]  # Convertir a lista y limitar

    finally:
        driver.quit()

Definimos la función para crear el dataset con imagenes de Lebron James y Messi

In [ ]:
def create_dataset(num_images_per_category=100):
    # Asegurar que exista la carpeta dataset
    create_directory(DATASET_PATH)

    # Categorías a descargar
    categories = {
        "Lionel Messi": num_images_per_category,
        "LeBron James": num_images_per_category
    }

    for category, num_images in categories.items():
        category_folder = os.path.join(DATASET_PATH, category.replace(" ", "_"))
        create_directory(category_folder)

        print(f"\nIniciando descarga de imágenes para la categoría: {category}")

        # Obtener URLs de imágenes
        image_urls = get_image_urls_using_bing(category, max_links_to_fetch=num_images*2)  # Obtener más URLs de las necesarias

        # Descargar imágenes
        downloaded_count = 0
        for i, url in enumerate(image_urls):
            if downloaded_count >= num_images:
                break

            # Generar un nombre basado en el hash de la URL para evitar duplicados
            img_name = f"{category.replace(' ', '_')}_{downloaded_count}"

            # Intentar descargar la imagen
            if download_image(url, category_folder, img_name):
                downloaded_count += 1
                print(f"Descargada imagen {downloaded_count}/{num_images} de '{category}'")

        print(f"Descarga completada. Se descargaron {downloaded_count} imágenes para {category}.")

    print("\nCreación del dataset completada. Estructura:")
    print(f"- {DATASET_PATH}/")
    for category in categories:
        category_name = category.replace(" ", "_")
        print(f"  - {category_name}/")
        print(f"    - {category_name}_0.jpg")
        print(f"    - {category_name}_1.jpg")
        print(f"    - ...")

In [ ]:
create_dataset(num_images_per_category=100)

El directorio 'C:\Users\ctorr\OneDrive\Documentos\Arturo\Arturo HW ITESO\6to semestre\Deep learning\Github\Proyecto 1 - DAE + VAE\dataset' ya existe.
El directorio 'C:\Users\ctorr\OneDrive\Documentos\Arturo\Arturo HW ITESO\6to semestre\Deep learning\Github\Proyecto 1 - DAE + VAE\dataset\Lionel_Messi' ya existe.

Iniciando descarga de imágenes para la categoría: Lionel Messi
Descargada imagen 1/100 de 'Lionel Messi'
Descargada imagen 2/100 de 'Lionel Messi'
Descargada imagen 3/100 de 'Lionel Messi'
Descargada imagen 4/100 de 'Lionel Messi'
Descargada imagen 5/100 de 'Lionel Messi'
Descargada imagen 6/100 de 'Lionel Messi'
Descargada imagen 7/100 de 'Lionel Messi'
Descargada imagen 8/100 de 'Lionel Messi'
Descargada imagen 9/100 de 'Lionel Messi'
Descargada imagen 10/100 de 'Lionel Messi'
Descargada imagen 11/100 de 'Lionel Messi'
Descargada imagen 12/100 de 'Lionel Messi'
Descargada imagen 13/100 de 'Lionel Messi'
Descargada imagen 14/100 de 'Lionel Messi'
Descargada imagen 15/100 de 'L

In [1]:
# Clona el repositorio
!git clone https://github.com/arturotowers/Deep_learning.git

Cloning into 'Deep_learning'...
remote: Enumerating objects: 450, done.
remote: Counting objects: 100% (450/450), done.
remote: Compressing objects: 100% (438/438), done.
remote: Total 450 (delta 15), reused 427 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (450/450), 4.72 MiB | 9.80 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [6]:
%cd Deep_learning

[Errno 2] No such file or directory: 'Deep_learning'
/content/Deep_learning


In [7]:
!git checkout Dataset_Creation


Branch 'Dataset_Creation' set up to track remote branch 'Dataset_Creation' from 'origin'.
Switched to a new branch 'Dataset_Creation'


## Procesamiento de imagenes
Importamos librerias para e preprocesamiento de imagenes y definimos la ruta y donde guardaremos las imagenes preprocesadas

In [8]:
import os
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Ruta de origen de las imágenes
source_dir = "/content/Deep_learning/Proyecto 1 - DAE + VAE/dataset"
# Carpeta destino para guardar las imágenes preprocesadas
dest_dir = "/content/Deep_learning/Proyecto 1 - DAE + VAE/processed_dataset"

Creamos la estructura para los splits y nos aseguramos de que la carpeta exista

In [9]:
splits = ['train', 'val', 'test']
split_ratios = {'train': 0.7, 'val': 0.15, 'test': 0.15}
os.makedirs(dest_dir, exist_ok=True)

In [10]:
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if os.path.isdir(class_path):
        # Buscar imágenes (se asume extensión .jpg; se puede agregar .png si es necesario)
        image_paths = glob.glob(os.path.join(class_path, "*.jpg"))
        if len(image_paths) == 0:
            image_paths = glob.glob(os.path.join(class_path, "*.png"))

        # Dividir las imágenes en train, val y test
        train_files, temp_files = train_test_split(image_paths, test_size=(1 - split_ratios['train']), random_state=42)
        val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)  # Dividir equitativamente entre validación y prueba

        split_files = {'train': train_files, 'val': val_files, 'test': test_files}

        # Procesar y guardar cada imagen en el directorio correspondiente
        for split in splits:
            split_class_dir = os.path.join(dest_dir, split, class_name)
            os.makedirs(split_class_dir, exist_ok=True)
            for file_path in split_files[split]:
                try:
                    # Abrir imagen y convertir a RGB
                    image = Image.open(file_path).convert('RGB')
                    # Redimensionar la imagen a un tamaño fijo (por ejemplo, 256x256)
                    image = image.resize((256, 256))
                    # Convertir a array y normalizar (valores de 0 a 1)
                    image_array = np.array(image, dtype=np.float32) / 255.0
                    # Guardar el array normalizado como .npy
                    base_name = os.path.basename(file_path)
                    file_name = os.path.splitext(base_name)[0] + ".npy"
                    np.save(os.path.join(split_class_dir, file_name), image_array)
                except Exception as e:
                    print(f"Error procesando {file_path}: {e}")

print("Preprocesamiento y organización completados.")

Preprocesamiento y organización completados.


- Redimensión: Se establece un tamaño fijo (256×256)
- Normalización: Se convierte cada imagen a un array de tipo float32 y se divide entre 255 para tener valores en [0, 1].
- Formato de guardado: Se guarda cada imagen preprocesada en formato .npy para facilitar su carga en modelos de deep learning.
- División de datos: Se usa train_test_split de scikit-learn para obtener una división balanceada.

In [11]:
!git add .
!git commit -m "Creamos el procesamiento de datos de nuestras imagenes"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@e9dadf45a5a8.(none)')


In [13]:
!git config --global user.name "arturotowers"

[Errno 2] No such file or directory: 'Deep_learning'
/content/Deep_learning
